In [1]:
import os
import glob
import pandas as pd
# Get all CSV files from the raw directory
raw_data_path = "../data/processed"
csv_files = glob.glob(os.path.join(raw_data_path, "*.csv"))

# Create a list of DataFrames, each containing a single CSV
dataframes = []
file_names = []

for csv_file in csv_files:
    # Extract filename without extension for reference
    file_name = os.path.basename(csv_file).replace('.csv', '')
    
    # Load CSV into DataFrame
    df = pd.read_csv(csv_file)
    
    # Store DataFrame and filename
    dataframes.append(df)
    file_names.append(file_name)
    
    print(f"Loaded {file_name}: {df.shape[0]} rows, {df.shape[1]} columns")

print(f"\nTotal DataFrames created: {len(dataframes)}")


df_dict = dict(zip(file_names, dataframes))
print(f"\nAvailable DataFrames: {list(df_dict.keys())}")

Loaded accounts_cleaned: 1635 rows, 6 columns
Loaded account_statuses: 3 rows, 2 columns
Loaded account_types: 5 rows, 2 columns
Loaded addresses_cleaned: 1210 rows, 4 columns
Loaded branches_cleaned: 50 rows, 3 columns
Loaded customers_cleaned: 1058 rows, 6 columns
Loaded customer_types: 3 rows, 2 columns
Loaded loans_cleaned: 316 rows, 7 columns
Loaded loan_statuses: 3 rows, 2 columns
Loaded transactions_cleaned: 48042 rows, 8 columns
Loaded transaction_types: 4 rows, 2 columns

Total DataFrames created: 11

Available DataFrames: ['accounts_cleaned', 'account_statuses', 'account_types', 'addresses_cleaned', 'branches_cleaned', 'customers_cleaned', 'customer_types', 'loans_cleaned', 'loan_statuses', 'transactions_cleaned', 'transaction_types']


In [2]:
df_transactions=df_dict['transactions_cleaned']
df_transaction_types=df_dict['transaction_types']
df_account_statuses=df_dict['account_statuses']
df_account_types=df_dict['account_types']
df_accounts=df_dict['accounts_cleaned']
df_branches=df_dict['branches_cleaned']
df_customer_types=df_dict['customer_types']
df_customers=df_dict['customers_cleaned']
df_loan_statuses=df_dict['loan_statuses']
df_loans=df_dict['loans_cleaned']
df_addresses=df_dict['addresses_cleaned']

In [ ]:
import pandas as pd

def build_enriched_transactions(transactions, transaction_types,
                                accounts, account_types, account_statuses,
                                customers, customer_types, addresses,
                                branches, loans, loan_statuses):
    # === STEP 1: Transaction types ===
    tx = transactions.merge(
        transaction_types,
        on="TransactionTypeID",
        how="left"
    ).rename(columns={"TypeName": "TransactionTypeName"})

    # === STEP 2: Origin account details ===
    tx = tx.merge(
        accounts.add_prefix("Origin_"),
        left_on="AccountOriginID",
        right_on="Origin_AccountID",
        how="left"
    )
    tx = tx.merge(
        account_types.add_prefix("Origin_"),
        left_on="Origin_AccountTypeID",
        right_on="Origin_AccountTypeID",
        how="left"
    ).rename(columns={"Origin_TypeName": "Origin_AccountType"})
    tx = tx.merge(
        account_statuses.add_prefix("Origin_"),
        left_on="Origin_AccountStatusID",
        right_on="Origin_AccountStatusID",
        how="left"
    ).rename(columns={"Origin_StatusName": "Origin_AccountStatus"})

    # === STEP 3: Destination account details ===
    tx = tx.merge(
        accounts.add_prefix("Dest_"),
        left_on="AccountDestinationID",
        right_on="Dest_AccountID",
        how="left"
    )
    tx = tx.merge(
        account_types.add_prefix("Dest_"),
        left_on="Dest_AccountTypeID",
        right_on="Dest_AccountTypeID",
        how="left"
    ).rename(columns={"Dest_TypeName": "Dest_AccountType"})
    tx = tx.merge(
        account_statuses.add_prefix("Dest_"),
        left_on="Dest_AccountStatusID",
        right_on="Dest_AccountStatusID",
        how="left"
    ).rename(columns={"Dest_StatusName": "Dest_AccountStatus"})

    # === STEP 4: Customer info (origin & dest) ===
    # Origin customer
    tx = tx.merge(
        customers.add_prefix("Origin_"),
        left_on="Origin_CustomerID",
        right_on="Origin_CustomerID",
        how="left"
    )
    tx = tx.merge(
        customer_types.add_prefix("Origin_"),
        left_on="Origin_CustomerTypeID",
        right_on="Origin_CustomerTypeID",
        how="left"
    ).rename(columns={"Origin_TypeName": "Origin_CustomerType"})
    tx = tx.merge(
        addresses.add_prefix("Origin_Addr_"),
        left_on="Origin_AddressID",
        right_on="Origin_Addr_AddressID",
        how="left"
    )

    # Dest customer
    tx = tx.merge(
        customers.add_prefix("Dest_"),
        left_on="Dest_CustomerID",
        right_on="Dest_CustomerID",
        how="left"
    )
    tx = tx.merge(
        customer_types.add_prefix("Dest_"),
        left_on="Dest_CustomerTypeID",
        right_on="Dest_CustomerTypeID",
        how="left"
    ).rename(columns={"Dest_TypeName": "Dest_CustomerType"})
    tx = tx.merge(
        addresses.add_prefix("Dest_Addr_"),
        left_on="Dest_AddressID",
        right_on="Dest_Addr_AddressID",
        how="left"
    )

    # === STEP 5: Branch info ===
    tx = tx.merge(
        branches.add_prefix("Branch_"),
        left_on="BranchID",
        right_on="Branch_BranchID",
        how="left"
    )
    tx = tx.merge(
        addresses.add_prefix("Branch_Addr_"),
        left_on="Branch_AddressID",
        right_on="Branch_Addr_AddressID",
        how="left"
    )

    # === STEP 6: Loan numeric features ===
    loan_features = loans.groupby("AccountID").agg({
        "LoanID": "count",
        "PrincipalAmount": "sum",
        "InterestRate": "mean"
    }).reset_index().rename(columns={
        "LoanID": "LoanCount",
        "PrincipalAmount": "TotalPrincipal",
        "InterestRate": "AvgInterestRate"
    })

    # === STEP 7: Loan status counts ===
    loan_status_features = loans.merge(
        loan_statuses, on="LoanStatusID", how="left"
    )
    loan_status_pivot = (
        loan_status_features
        .groupby(["AccountID", "StatusName"])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )
    # Example: columns = ["AccountID", "Active", "Paid Off", "Overdue"]

    # Merge origin + destination loan features
    tx = tx.merge(
        loan_features.add_prefix("Origin_"),
        left_on="Origin_AccountID",
        right_on="Origin_AccountID",
        how="left"
    ).merge(
        loan_features.add_prefix("Dest_"),
        left_on="Dest_AccountID",
        right_on="Dest_AccountID",
        how="left"
    )

    # Merge origin + destination loan statuses
    tx = tx.merge(
        loan_status_pivot.add_prefix("Origin_LoanStatus_"),
        left_on="Origin_AccountID",
        right_on="Origin_LoanStatus_AccountID",
        how="left"
    ).merge(
        loan_status_pivot.add_prefix("Dest_LoanStatus_"),
        left_on="Dest_AccountID",
        right_on="Dest_LoanStatus_AccountID",
        how="left"
    )

    return tx



In [ ]:
df_transactions,df_transaction_types,df_accounts,df_account_types,df_account_statuses,df_customers,df_customer_types,df_addresses,df_branches,df_loans,df_loan_statusesdf_final=build_enriched_transactions()

In [12]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48042 entries, 0 to 48041
Data columns (total 66 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   TransactionID                48042 non-null  int64  
 1   AccountOriginID              48042 non-null  int64  
 2   AccountDestinationID         48042 non-null  int64  
 3   TransactionTypeID            48042 non-null  int64  
 4   Amount                       48042 non-null  float64
 5   TransactionDate              48042 non-null  object 
 6   BranchID                     48042 non-null  int64  
 7   Description                  48042 non-null  object 
 8   TransactionTypeName          48042 non-null  object 
 9   Origin_AccountID             47581 non-null  float64
 10  Origin_CustomerID            47581 non-null  float64
 11  Origin_AccountTypeID         47581 non-null  float64
 12  Origin_AccountStatusID       47581 non-null  float64
 13  Origin_Balance  

In [14]:
df_final.head(50)

,TransactionID,AccountOriginID,AccountDestinationID,TransactionTypeID,Amount,TransactionDate,BranchID,Description,TransactionTypeName,Origin_AccountID,...,Dest_TotalPrincipal,Dest_AvgInterestRate,Origin_LoanStatus_AccountID,Origin_LoanStatus_Active,Origin_LoanStatus_Overdue,Origin_LoanStatus_Paid Off,Dest_LoanStatus_AccountID,Dest_LoanStatus_Active,Dest_LoanStatus_Overdue,Dest_LoanStatus_Paid Off
0,3000000,201241,200847,3,2984.02,2023-12-29 10:00:00.000000,26,Transaction 0,Transfer,201241.0,...,NaN,NaN,201241.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
1,3000001,201103,200262,3,4713.48,2023-05-12 02:00:00.000000,23,Transaction 1,Transfer,201103.0,...,59673.83,0.14110,201103.0,2.0,0.0,0.0,200262.0,1.0,0.0,0.0
2,3000002,201563,200589,3,1600.23,2020-07-14 17:00:00.000000,5,Transaction 2,Transfer,201563.0,...,NaN,NaN,201563.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
3,3000003,200512,200404,1,4279.61,2023-09-09 07:00:00.000000,36,Transaction 3,Deposit,200512.0,...,97362.97,0.11450,200512.0,1.0,0.0,0.0,200404.0,1.0,0.0,0.0
4,3000004,201107,201408,2,4125.32,2022-05-29 02:00:00.000000,44,Transaction 4,Withdrawal,201107.0,...,NaN,NaN,201107.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
5,3000005,200353,201615,3,2445.17,2023-07-14 03:00:00.000000,26,Transaction 5,Transfer,200353.0,...,NaN,NaN,200353.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
6,3000006,200822,200492,3,3592.37,2020-11-23 23:00:00.000000,14,Transaction 6,Transfer,200822.0,...,NaN,NaN,200822.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
7,3000007,200255,200152,1,3291.83,2020-04-16 21:00:00.000000,40,Transaction 7,Deposit,200255.0,...,NaN,NaN,200255.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
8,3000008,200379,200993,1,2965.03,2022-10-10 14:00:00.000000,47,Transaction 8,Deposit,200379.0,...,NaN,NaN,200379.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
9,3000009,201103,200376,3,611.83,2023-01-10 11:00:00.000000,47,Transaction 9,Transfer,201103.0,...,72508.87,0.05520,201103.0,2.0,0.0,0.0,200376.0,1.0,0.0,0.0


In [15]:
import pandas as pd
import numpy as np

def compute_transaction_features(tx):
    """
    Compute initial anomaly detection features from enriched transaction DataFrame
    """
    df = tx.copy()

    # --- Transaction Amount Features ---
    df['Amount_to_OriginBalance'] = df['Amount'] / df['Origin_Balance'].replace(0, np.nan)
    df['Amount_to_DestBalance'] = df['Amount'] / df['Dest_Balance'].replace(0, np.nan)

    # --- Account Features ---
    # Encode account status flags
    df['Origin_AccountInactive'] = df['Origin_AccountStatus'].isin(['Inactive', 'Closed']).astype(int)
    df['Dest_AccountInactive'] = df['Dest_AccountStatus'].isin(['Inactive', 'Closed']).astype(int)

    # --- Customer Features ---
    # Age
    df['Origin_DateOfBirth'] = pd.to_datetime(df['Origin_DateOfBirth'], errors='coerce')
    df['Dest_DateOfBirth'] = pd.to_datetime(df['Dest_DateOfBirth'], errors='coerce')
    today = pd.Timestamp.today()
    df['Origin_Age'] = (today - df['Origin_DateOfBirth']).dt.days // 365
    df['Dest_Age'] = (today - df['Dest_DateOfBirth']).dt.days // 365

    # --- Loan Features ---
    # Loan leverage ratios
    df['Origin_LoanLeverage'] = df['Origin_TotalPrincipal'] / df['Origin_Balance'].replace(0, np.nan)
    df['Dest_LoanLeverage'] = df['Dest_TotalPrincipal'] / df['Dest_Balance'].replace(0, np.nan)

    # Overdue loan flags
    df['Origin_OverdueLoans'] = df.get('Origin_LoanStatus_Overdue', 0)
    df['Dest_OverdueLoans'] = df.get('Dest_LoanStatus_Overdue', 0)

    # --- Transaction Time Features ---
    df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')
    df['TransactionHour'] = df['TransactionDate'].dt.hour
    df['TransactionWeekday'] = df['TransactionDate'].dt.dayofweek  # 0 = Monday

    # --- Optional flags ---
    #df['LargeTransferFlag'] = (df['Amount_to_OriginBalance'] > 0.5).astype(int)

    # Fill NaNs for numeric features (optional)
    numeric_cols = [
        'Amount_to_OriginBalance', 'Amount_to_DestBalance',
        'Origin_Balance', 'Dest_Balance',
        'Origin_LoanCount', 'Dest_LoanCount',
        'Origin_TotalPrincipal', 'Dest_TotalPrincipal',
        'Origin_AvgInterestRate', 'Dest_AvgInterestRate',
        'Origin_LoanLeverage', 'Dest_LoanLeverage',
        'Origin_OverdueLoans', 'Dest_OverdueLoans',
        'Origin_Age', 'Dest_Age'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    return df


In [17]:
df_final1=compute_transaction_features(df_final)

In [18]:
df_final1.head()

,TransactionID,AccountOriginID,AccountDestinationID,TransactionTypeID,Amount,TransactionDate,BranchID,Description,TransactionTypeName,Origin_AccountID,...,Origin_AccountInactive,Dest_AccountInactive,Origin_Age,Dest_Age,Origin_LoanLeverage,Dest_LoanLeverage,Origin_OverdueLoans,Dest_OverdueLoans,TransactionHour,TransactionWeekday
0,3000000,201241,200847,3,2984.02,2023-12-29 10:00:00,26,Transaction 0,Transfer,201241.0,...,0,1,62.0,58.0,0.945079,0.000000,1.0,0.0,10,4
1,3000001,201103,200262,3,4713.48,2023-05-12 02:00:00,23,Transaction 1,Transfer,201103.0,...,0,0,52.0,36.0,6.365179,0.843718,0.0,0.0,2,4
2,3000002,201563,200589,3,1600.23,2020-07-14 17:00:00,5,Transaction 2,Transfer,201563.0,...,0,1,48.0,42.0,0.356985,0.000000,0.0,0.0,17,1
3,3000003,200512,200404,1,4279.61,2023-09-09 07:00:00,36,Transaction 3,Deposit,200512.0,...,0,0,51.0,54.0,1.240016,2.984420,0.0,0.0,7,5
4,3000004,201107,201408,2,4125.32,2022-05-29 02:00:00,44,Transaction 4,Withdrawal,201107.0,...,0,0,50.0,0.0,0.658676,0.000000,0.0,0.0,2,6
